# 第二十章 属性描述符

## 20.1描述符示例：验证属性

### 20.1.1 LineItem类第三版：一个简单的描述符

实现了__get__、__set__、__delete__方法的类是描述符。描述符的用法是，创建一个实例，作为另一个类的类属性。

![chapter20-1](image/chapter20-1.jpg)


![chapter20-1-2](image/chapter20-1-2.jpg)

In [1]:
#描述符类
class Quantity:
    
    def __init__(self,storage_name):
        self.storage_name = storage_name
    
    def __set__(self,instance,value):#self为描述符类实例，instance为托管类实例
                                     
        if value >0:
            instance.__dict__[self.storage_name]=value#必须直接处理托管类实例的__dict__属性
                                                      # 如果使用内置的setattr函数，会再次触发__set__方法，导致无限递归
        else:
            raise ValueError('value must be >0')

In [2]:
#托管类
class LineItem:
    weight=Quantity('weight')
    price=Quantity('price')
    
    def __init__(self,description,weight,price):
        self.description=description
        self.weight=weight #会调用Quantity的__set__方法
        self.price=price
    
    def subtotal(self):
        return self.weight*self.price

In [3]:
truffle=LineItem('white truffle',100,0)

ValueError: value must be >0

![chapter20-1-3](image/chapter20-1-3.png)

![chapter20-1-4](image/chapter20-1-4.jpg)

### 20.1.2 LineItem类第四版：自动获取储存属性的名称

![chapter20-2](image/chapter20-2.jpg)

In [10]:
#每个Quantity描述符都有一个独一无二的storage_name
class Quantity:
    __counter=0
    
    def __init__(self):
        cls=self.__class__
        prefix=cls.__name__
        index=cls.__counter
        self.storage_name='_{}#{}'.format(prefix,index)
        cls.__counter+=1
    
    def __get__(self, instance, owner):
        return getattr(instance,self.storage_name)
    
    def __set__(self, instance, value):
        if value>0:
            setattr(instance,self.storage_name,value)#不会调用特性了
        else:
            raise  ValueError('value must be >0')
    
class LineItem:
    weight=Quantity()
    price=Quantity()
    
    def __init__(self,description,weight,price):
        self.description=description
        self.weight=weight #会调用Quantity的__set__方法
        self.price=price
    
    def subtotal(self):
        return self.weight*self.price

In [11]:
cocount=LineItem('Brazilian cocount',20,17.95)

In [12]:
cocount.weight,cocount.price

(20, 17.95)

In [14]:
getattr(cocount,'_Quantity#0'),getattr(cocount,'_Quantity#1')

(20, 17.95)

In [15]:
cocount.__dict__

{'_Quantity#0': 20, '_Quantity#1': 17.95, 'description': 'Brazilian cocount'}

In [16]:
cocount2=LineItem('Brazilian ',2,1)

In [17]:
cocount2.weight,cocount2.price

(2, 1)

In [20]:
getattr(cocount,'weight')

20

![chapter20-2-2](image/chapter20-2-2.png)

In [21]:
LineItem.weight

AttributeError: 'NoneType' object has no attribute '_Quantity#0'

In [22]:
'''
__get__有三个参数，self/instance/owner，owner参数是托管类（如LineItem）的引用，通过描述符从托管类中获取属性时用到。
如果使用LineItem.weight从类中获取托管属性，描述符的__get__方法接收到的instance参数值是None。
因此会抛出AttributeError
'''

'\n__get__有三个参数，self/instance/owner，owner参数是托管类（如LineItem）的引用，通过描述符从托管类中获取属性时用到。\n如果使用LineItem.weight从类中获取托管属性，描述符的__get__方法接收到的instance参数值是None。\n因此会抛出AttributeError\n'

In [23]:
#通过托管类调用时，__get__方法返回描述符的引用
#每个Quantity描述符都有一个独一无二的storage_name
class Quantity:
    __counter=0
    
    def __init__(self):
        cls=self.__class__
        prefix=cls.__name__
        index=cls.__counter
        self.storage_name='_{}#{}'.format(prefix,index)
        cls.__counter+=1
    
    def __get__(self, instance, owner):
        if instance is None:
            return self#返回描述符自身
        else:
            return getattr(instance,self.storage_name)
    
    def __set__(self, instance, value):
        if value>0:
            setattr(instance,self.storage_name,value)#不会调用特性了
        else:
            raise  ValueError('value must be >0')
    
class LineItem:
    weight=Quantity()
    price=Quantity()
    
    def __init__(self,description,weight,price):
        self.description=description
        self.weight=weight #会调用Quantity的__set__方法
        self.price=price
    
    def subtotal(self):
        return self.weight*self.price

In [24]:
LineItem.price

In [25]:
cocount2=LineItem('Brazilian ',2,1)

In [26]:
cocount2.price

1

通常，我们不会再使用描述符的模块中定义描述符，而是在一个单独的实用工具中定义，以便在整个应用中使用——如果开发的是框架，甚至会在多个应用中使用，

In [27]:
import sys
sys.path.insert(0,r'D:\WORKSPACE2\python35\python8.25\fluent_python')

In [28]:
import model_v4c as model

class LineItem:
    weight=model.Quantity()
    price=model.Quantity()
    
    def __init__(self,description,weight,price):
        self.description=description
        self.weight=weight #会调用Quantity的__set__方法
        self.price=price
    
    def subtotal(self):
        return self.weight*self.price  
    

特性工厂函数与描述符类比较

In [ ]:
#quantity特性工厂函数的实现
def quantity():
    try:
        quantity.counter+=1 #不能依靠类属性在多次调用之间共享counter，因此把它定义为quantity函数自身的属性
    except AttributeError:
        quantity.counter=0
    
    storage_name='_{}:{}'.format('quantity',quantity.counter) #局部变量storage_name，借助闭包保持它的值，供后面两个函数使用
        
    
    def qty_getter(instance):
        return instance.__dict__[storage_name]#引用lestorage_name,闭包
    
    def qty_setter(instance,value):
        if value>0:
            instance.__dict__[storage_name]=value
        else:
            raise ValueError('value must be >0')
    return property(qty_getter,qty_setter)

特性工厂函数模式较简单，可是描述符类方式更易扩展

### 20.1.3 LineItem类第五版：一种新型描述符

In [29]:
import abc


class AutoStorage:  # <1>
    __counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)

    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)  # <2>


class Validated(abc.ABC, AutoStorage):  # <3>

    def __set__(self, instance, value):
        value = self.validate(instance, value)  # <4>
        super().__set__(instance, value)  # <5>

    @abc.abstractmethod
    def validate(self, instance, value):  # <6>
        """return validated value or raise ValueError"""


class Quantity(Validated):  # <7>
    """a number greater than zero"""

    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value


class NonBlank(Validated):
    """a string with at least one non-space character"""

    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value  # <8>

In [30]:
import model_v5 as model
class LineItem:
    description=model.NonBlank()
    weight=model.Quantity()
    price=model.Quantity()
    
    def __init__(self,description,weight,price):
        self.description=description
        self.weight=weight #会调用Quantity的__set__方法
        self.price=price
    
    def subtotal(self):
        return self.weight*self.price  

In [31]:
a=LineItem('',1,2)

ValueError: value cannot be empty or blank

20.2 覆盖型与非覆盖型描述符对比

In [1]:
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return '<{} object>'.format(cls_name(obj))

def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))


### essential classes for this example ###

class Overriding:  # <1>
    """a.k.a. data descriptor or enforced descriptor"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)  # <2>

    def __set__(self, instance, value):
        print_args('set', self, instance, value)


class OverridingNoGet:  # <3>
    """an overriding descriptor without ``__get__``"""

    def __set__(self, instance, value):
        print_args('set', self, instance, value)


class NonOverriding:  # <4>
    """a.k.a. non-data or shadowable descriptor"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)


class Managed:  # <5>
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()

    def spam(self):  # <6>
        print('-> Managed.spam({})'.format(display(self)))

# END DESCR_KINDS

In [2]:
import sys
sys.path.insert(0,r'D:\WORKSPACE2\python35\python8.25\fluent_python')

### 20.2.1  覆盖型描述符

In [3]:
obj=Managed()
obj.over #触发描述符__get__，第二个参数是托管实例

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [4]:
Managed.over #触发描述符__get__，第二个参数（instance)为None

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [5]:
obj.over=7 #触发__set__方法，最后一个参数是7

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [6]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [7]:
obj.__dict__['over'] = 8 #跳过描述符，直接通过obj.__dict__赋值

In [8]:
vars(obj)

{'over': 8}

In [9]:
obj.over  #即使是名为over的实例属性，Managed.over描述符仍会覆盖读取obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


###  20.2.2 没有__get__方法的覆盖型描述符

In [10]:
obj.over_no_get #描述符没有__get__方法，因此，obj.over_no_get从类中获取描述符实例

In [11]:
Managed.over_no_get

In [12]:
obj.over_no_get=7 #触发__set__方法

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [13]:
obj.over_no_get

In [14]:
obj.__dict__['over_no_get'] = 9

In [15]:
obj.over_no_get #实例属性覆盖描述符，但是只有读操作如此

9

In [16]:
obj.over_no_get=7 #赋值时，仍经过描述符的__set__处理

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [17]:
obj.over_no_get #读取时，只要有同名的实例属性，描述符就会被覆盖

9

###  20.2.3  非覆盖型描述符

没有实现__set__方法的描述符是非覆盖型描述符。如果设置了同名的实例属性，描述符会被遮盖。

In [18]:
obj=Managed()
obj.non_over #触发__get__

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [19]:
obj.non_over=7 #非覆盖型描述符没有干涉赋值操作的__set__方法

In [21]:
obj.non_over #现在obj有个名为non_over的实例属性，把Managed类的同名描述符属性遮盖掉

7

In [22]:
Managed.non_over

-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)


In [23]:
del obj.non_over

In [24]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


### 20.2.4  在类中覆盖描述符

不管描述符是不是覆盖型，为类属性赋值都能覆盖描述符。这是一种猴子补丁技术。

In [25]:
obj=Managed()
Managed.over=1 #覆盖类中的描述符属性

In [26]:
Managed.over_no_get=2
Managed.non_over=3

In [28]:
obj.over,obj.over_no_get,obj.non_over #描述符真的不见了

(1, 2, 3)

上例揭示了读写属性的另一种不对等：读类属性的操作可以由依附在托管类上定义有__get__方法的描述符处理，但是写类属性的操作不会由依附在托管类上定义有__set__方法的描述符处理

## 20.3 方法是描述符

在类中定义的函数属于绑定方法（bound method），因为用户定义的函数都有__get__方法，所以依附到类上时，就相当于描述符。

In [29]:
obj=Managed()  
obj.spam #obj.spam获取的是绑定方法对象

<bound method Managed.spam of <__main__.Managed object at 0x000001E4F2F0EBE0>>

In [30]:
Managed.spam

<function __main__.Managed.spam>

In [31]:
obj.spam=7 #如果为obj.spam赋值，会遮盖类属性，导致无法通过obj实例访问spam方法

In [32]:
obj.spam

7

函数没有实现__set__方法，因此是非覆盖型描述符

In [33]:
import collections

class Text(collections.UserString):
    def __repr__(self):
        return 'Text({!r})'.format(self.data)
    
    def reverse(self):
        return self[::-1]

In [34]:
word=Text('forward')
word

Text('forward')

In [35]:
word.reverse()

Text('drawrof')

In [36]:
Text.reverse(Text('backward')) #在类上调用方法相当于调用函数

Text('drawkcab')

In [37]:
type(Text.reverse),type(word.reverse) #类型不同！

(function, method)

In [38]:
list(map(Text.reverse, ['repaid', (10, 20, 30), Text('stressed')])) #Text.reverse相当于函数

['diaper', (30, 20, 10), Text('desserts')]

In [40]:
Text.reverse.__get__(word)  #函数都是非覆盖型描述符。在函数上调用__get__方法时传入实例，得到的是绑定到那个实例上的方法

<bound method Text.reverse of Text('forward')>

In [41]:
Text.reverse.__get__(None,Text) #调用方法时如果instance参数的值是None，那么得到的是函数本身

<function __main__.Text.reverse>

In [42]:
word.reverse #其实会调用Text.reverse.__get__(word),返回对应的绑定方法

<bound method Text.reverse of Text('forward')>

In [43]:
word.reverse.__self__  #绑定方法对象有个__self__属性，其值是调用这个方法的实例引用

Text('forward')

In [44]:
word.reverse.__func__ is Text.reverse

True